<a href="https://colab.research.google.com/github/yeoshuheng/freeCodeCamp-Machine-Learning-With-Python-Solutions/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep = '\t', header = None)
df_test = pd.read_csv(test_file_path, sep = '\t', header = None)
df_train.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# 0 = ham, 1 = spam
df_train[0] = le.fit_transform(df_train[0])
df_train.head()

In [ ]:
df_test[0] = le.fit_transform(df_test[0])
df_test.head()

In [ ]:
df_test.dtypes

In [ ]:
train_y = df_train[0].to_numpy()
slice_x  = df_train[1].to_numpy()
train_ds = tf.data.Dataset.from_tensor_slices((slice_x, train_y))

In [ ]:
test_y = df_test[0].to_numpy()
test_slice_x = df_test[1].to_numpy()
test_ds = tf.data.Dataset.from_tensor_slices((test_slice_x, test_y))

In [ ]:
#batching the tensorslices

BUFFER_SIZE = 100
BATCH_SIZE = 64

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_x.element_spec

In [ ]:
#text encoder
encoder = tf.keras.layers.TextVectorization(
    max_tokens = 1000,
    output_mode = 'int',
    output_sequence_length = 1000
)
encoder.adapt(train_ds.map(lambda text, label : text ))

In [ ]:
word_list = np.array(encoder.get_vocabulary())
print(word_list)

In [ ]:
model = keras.Sequential([
                       encoder,
                       tf.keras.layers.Embedding(input_dim = len(encoder.get_vocabulary()), output_dim = 64, mask_zero = True),
                       tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                       tf.keras.layers.Dense(64, activation = 'relu'),
                       tf.keras.layers.Dense(1)
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
model.fit(train_ds, validation_data = test_ds, epochs = 10, validation_steps = 30)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  result = model.predict([pred_text])
  #pred_text needs to be in a list as model expects a nested list input
  probability = result[0][0]
  if probability >= 0.5:
    p_class = 'spam'
  else:
    p_class = 'ham'

  prediction = [probability, p_class]




  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
